In [ ]:
using Revise
using AllenCahn
using Plots

In [ ]:
# Construct the problem
a = 0; b = 1
ϵ = 0.1
nₓ = 9
nₜ = 100; t_max = 1
left_bc = NeumannBC(0); right_bc = NeumannBC(0)
u₀(x) = 1.
problem = AllenCahnProblem1D(a, b, ϵ, nₓ, nₜ, t_max, left_bc, right_bc, u₀);

In [ ]:
# Check that the problem constructor works
println("a: $(problem.a), b: $(problem.b), ϵ: $(problem.ϵ), Δx: $(problem.Δx) Δt: $(problem.Δt)")
println("mesh: $(problem.x)")

In [ ]:
# Assembly and check
assemble_system!(problem);
println("A: $(problem.A)")
println("RHS: $(problem.rhs)")

In [ ]:
# Solve problem and plot solution vs. expected result
u = solve(problem, BackwardEulerMethod())
u_true(x) = 1.
plot(problem.x, u_true.(problem.x), label = "True Solution", ylims = (0, 2))
scatter!(problem.x, u, label = "Numerical Solution")

In [ ]:
# Use a test to check that the package passes
using Pkg
Pkg.test("AllenCahn")